In [1]:
import os
import pandas as pd
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
import numpy as np
import json

## Embeddings

In [1]:
import os
import pandas as pd
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
import numpy as np
import json

In [29]:
df = pd.read_csv(r'C:\Users\Hemish Savaliya\Downloads\combined_openai.csv')


In [30]:
df

,Unnamed: 0,worker,confidence,newsID,Article,category,summary,Headline
0,0,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,5.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,Fiance and I recently got infected with scabi...,19f with fiance 20m and roommate 19m- fiance a...
1,1,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,4.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,my fiance and I refuse to treat our room mate...,19f with fiance 20m and roommate 19m- fiance a...
2,2,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,"fiancé and I contracted scabies, roommate ref...",19f with fiance 20m and roommate 19m- fiance a...
3,3,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,"fiance and I are infected with scabies, room ...",19f with fiance 20m and roommate 19m- fiance a...
4,4,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,"Fiance and I contracted scabies, roommate ref...",19f with fiance 20m and roommate 19m- fiance a...
...,...,...,...,...,...,...,...,...
185240,185240,RMwrIV50cNusBthNvLs1wSNdqFpQAg,1.0,t3_kz1gv,Hello World of Reddit\n\nI'm a 21 year old fem...,AskReddit,I'm a 21year old female anemic taking iron su...,"I have iron deficient anemia, need help and/or..."
185241,185241,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,I don't know how to ask this without it soundi...,r/relationships,"Boyfriend talks very slowly, I feel frustrated...",Me [27 F] with my boyfriend [24 M] for four mo...
185242,185242,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,I honestly do not care anymore. \n\nExcluding ...,r/Advice,Medical field looks like its going to hell. Wh...,Almost every career in the medical field is sa...
185243,185243,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,Someone I consider to be a best friend and I a...,r/relationship_advice,"Met a guy on reddit, became best friends, met,...",I'm [26/f] in US and my best friend [32/m] is ...


In [31]:
# Clean the 'summary' column to ensure all inputs are valid strings
df['summary'] = df['summary'].astype(str).str.strip()  # Convert to string and strip whitespace
df = df[df['summary'].str.len() > 0]  # Remove empty strings


In [32]:

# Extract the cleaned summaries
summaries = df['summary'].tolist()

In [108]:
# Create a directory for saving chunk embeddings
CHUNK_DIR = "./embeddings_chunks_with_newsID/"
os.makedirs(CHUNK_DIR, exist_ok=True)

In [10]:
import os
# Create a directory for saving chunk embeddings
CHUNK_DIR = "./embeddings_chunks_with_newsID/"
os.makedirs(CHUNK_DIR, exist_ok=True)
# Load BERT model and tokenizer
model = TFBertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to generate embeddings for a batch of text
def generate_embeddings(texts, model, tokenizer, pooling="cls"):
    inputs = tokenizer(texts, return_tensors="tf", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    if pooling == "cls":
        # Use [CLS] token embedding
        return outputs.last_hidden_state[:, 0, :].numpy()
    elif pooling == "mean":
        # Use mean pooling
        return tf.reduce_mean(outputs.last_hidden_state, axis=1).numpy()
    else:
        raise ValueError("Invalid pooling method. Use 'cls' or 'mean'.")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [34]:
# Split the DataFrame into chunks
chunk_size = 100  # Define chunk size
chunks = [df.iloc[i:i+chunk_size] for i in range(0, len(df), chunk_size)]


In [36]:
# Process each chunk and save embeddings with newsID
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}/{len(chunks)}...")
    
    # Extract summaries and newsIDs from the chunk
    batch_summaries = chunk['summary'].astype(str).tolist()
    batch_newsIDs = chunk['newsID'].tolist()
    
    # Generate embeddings for the chunk
    chunk_embeddings = generate_embeddings(batch_summaries, model, tokenizer)
    
    # Combine embeddings with newsID for tracking
    chunk_data = [{"newsID": newsID, "embedding": embedding} for newsID, embedding in zip(batch_newsIDs, chunk_embeddings)]
    
    # Save the chunk to a file
    chunk_file = os.path.join(CHUNK_DIR, f"chunk_{i}.npy")
    np.save(chunk_file, chunk_data)
    
    print(f"Chunk {i + 1} saved to {chunk_file}.")

print("All chunks processed and saved!")

Processing chunk 1/1853...


NameError: name 'generate_embeddings' is not defined

In [68]:
# Load all chunk embeddings
chunk_files = sorted(
    [f for f in os.listdir(CHUNK_DIR) if f.startswith("chunk_")],
    key=lambda x: int(x.split("_")[1].split(".")[0])
)


# Print the sorted order for verification
print("Files in sorted order:", chunk_files)

Files in sorted order: ['chunk_0.npy', 'chunk_1.npy', 'chunk_2.npy', 'chunk_3.npy', 'chunk_4.npy', 'chunk_5.npy', 'chunk_6.npy', 'chunk_7.npy', 'chunk_8.npy', 'chunk_9.npy', 'chunk_10.npy', 'chunk_11.npy', 'chunk_12.npy', 'chunk_13.npy', 'chunk_14.npy', 'chunk_15.npy', 'chunk_16.npy', 'chunk_17.npy', 'chunk_18.npy', 'chunk_19.npy', 'chunk_20.npy', 'chunk_21.npy', 'chunk_22.npy', 'chunk_23.npy', 'chunk_24.npy', 'chunk_25.npy', 'chunk_26.npy', 'chunk_27.npy', 'chunk_28.npy', 'chunk_29.npy', 'chunk_30.npy', 'chunk_31.npy', 'chunk_32.npy', 'chunk_33.npy', 'chunk_34.npy', 'chunk_35.npy', 'chunk_36.npy', 'chunk_37.npy', 'chunk_38.npy', 'chunk_39.npy', 'chunk_40.npy', 'chunk_41.npy', 'chunk_42.npy', 'chunk_43.npy', 'chunk_44.npy', 'chunk_45.npy', 'chunk_46.npy', 'chunk_47.npy', 'chunk_48.npy', 'chunk_49.npy', 'chunk_50.npy', 'chunk_51.npy', 'chunk_52.npy', 'chunk_53.npy', 'chunk_54.npy', 'chunk_55.npy', 'chunk_56.npy', 'chunk_57.npy', 'chunk_58.npy', 'chunk_59.npy', 'chunk_60.npy', 'chunk_61.

In [112]:
sample_chunk = np.load(os.path.join(CHUNK_DIR, "chunk_0.npy"), allow_pickle=True)
sample_chunk

array([[[-0.11807224, -0.10583329,  0.07036594, ..., -0.26158988,
          0.47443888,  0.46663532],
        [ 0.33639997, -0.21811119,  0.57311547, ..., -0.5226127 ,
          0.875703  , -0.35529125],
        [-0.08782338, -0.14394608,  0.21092536, ..., -0.41436797,
          0.36738858,  0.05532727],
        ...,
        [ 0.07259348,  0.07439183,  0.6857361 , ...,  0.07876956,
         -0.22769643,  0.21609   ],
        [-0.11391051, -0.07406129,  0.63413924, ..., -0.06625573,
         -0.12837431,  0.32239294],
        [ 0.22929937, -0.01686813,  0.7657648 , ...,  0.32139832,
          0.00698328,  0.05776851]],

       [[ 0.00599561,  0.54013413, -0.1943281 , ..., -0.24688995,
          0.4622362 ,  0.57881206],
        [ 0.15795024,  0.24428752,  0.04178084, ..., -0.1101455 ,
          0.91649735,  0.27452633],
        [ 0.46630323,  0.10853219,  0.20487502, ..., -0.32055986,
          0.64469093, -0.468925  ],
        ...,
        [ 0.30411482,  0.06800052,  0.02194718, ..., -

In [ ]:


all_embeddings = []

# Load and combine all chunks
for chunk_file in chunk_files:
    chunk_path = os.path.join(CHUNK_DIR, chunk_file)
    chunk_data = np.load(chunk_path, allow_pickle=True)
    all_embeddings.extend(chunk_data)

# Convert to a DataFrame
embedding_df = pd.DataFrame(all_embeddings)

# Merge with the original DataFrame on `newsID`
merged_df = pd.merge(df, embedding_df, on="newsID", how="left")




In [ ]:
merged_df

In [96]:
# Replace the 'summary' column with the 'embedding' column
merged_df['summary'] = merged_df['embedding']

# Drop the original 'summary' column

merged_df = merged_df.drop(columns=['embedding'])

merged_df


KeyError: 'embedding'

In [56]:
merged_df

NameError: name 'merged_df' is not defined

In [92]:
# Save the updated DataFrame
output_path = r"C:\Users\Hemish Savaliya\Documents\Chunks_embed\updated_merged_df_1.csv"
merged_df.to_csv(output_path, index=False)
print(f"Updated merged DataFrame saved successfully at {output_path}!")

Updated merged DataFrame saved successfully at C:\Users\Hemish Savaliya\Documents\Chunks_embed\updated_merged_df.csv!


## final merge

In [2]:
df = pd.read_csv(r'C:\Users\Hemish Savaliya\Downloads\combined_openai.csv')
grouped_path = r"C:\Users\Hemish Savaliya\Documents\Chunks_embed\grouped_with_embeddings.csv"

# Load grouped DataFrame
grouped = pd.read_csv(grouped_path)


In [3]:
df

,Unnamed: 0,worker,confidence,newsID,Article,category,summary,Headline
0,0,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,5.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,Fiance and I recently got infected with scabi...,19f with fiance 20m and roommate 19m- fiance a...
1,1,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,4.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,my fiance and I refuse to treat our room mate...,19f with fiance 20m and roommate 19m- fiance a...
2,2,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,"fiancé and I contracted scabies, roommate ref...",19f with fiance 20m and roommate 19m- fiance a...
3,3,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,"fiance and I are infected with scabies, room ...",19f with fiance 20m and roommate 19m- fiance a...
4,4,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,"Fiance and I contracted scabies, roommate ref...",19f with fiance 20m and roommate 19m- fiance a...
...,...,...,...,...,...,...,...,...
185240,185240,RMwrIV50cNusBthNvLs1wSNdqFpQAg,1.0,t3_kz1gv,Hello World of Reddit\n\nI'm a 21 year old fem...,AskReddit,I'm a 21year old female anemic taking iron su...,"I have iron deficient anemia, need help and/or..."
185241,185241,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,I don't know how to ask this without it soundi...,r/relationships,"Boyfriend talks very slowly, I feel frustrated...",Me [27 F] with my boyfriend [24 M] for four mo...
185242,185242,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,I honestly do not care anymore. \n\nExcluding ...,r/Advice,Medical field looks like its going to hell. Wh...,Almost every career in the medical field is sa...
185243,185243,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,Someone I consider to be a best friend and I a...,r/relationship_advice,"Met a guy on reddit, became best friends, met,...",I'm [26/f] in US and my best friend [32/m] is ...


In [4]:
grouped

,group_id,worker,newsID,Article,Headline
0,1,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,t3_4l0bal,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
1,2,dL2Aff39094mOhTKsLkJg8sCCFbirW,t3_4iqygu,"[0.12550995, -0.014912054, -0.20902874, -0.778...","[-0.16134378, 0.09565151, -0.006751992, -0.368..."
2,3,dL2Aff39094mOhTKsLkJg8sCCFbirW,t3_2dmjeu,"[0.21124741, -0.3693796, 0.2616318, -0.2376193...","[-0.20285274, 0.16432627, 0.019520625, -0.1310..."
3,4,dL2Aff39094mOhTKsLkJg8sCCFbirW,t3_2k3zi1,"[0.19669813, -0.27256715, -0.20540851, -0.3430...","[-0.15736824, -0.09334688, -0.22660448, 0.1772..."
4,5,RMwrIV50cNusBthNvLs1wSNdqFpQAg,t3_zjaee,"[0.11704462, -0.37076092, 0.0050954893, -0.453...","[-0.20783931, -0.27736032, -0.5712667, -0.0517..."
...,...,...,...,...,...
61341,61342,RMwrIV50cNusBthNvLs1wSNdqFpQAg,t3_kz1gv,"[0.26689363, -0.29950368, 0.18191998, -0.29945...","[0.038773768, 0.12896419, 0.0573383, -0.503152..."
61342,61343,soP7Okyp0e1WdYxQC9IldpwnznT6V1,NaN,"[0.11833226, -0.15423085, 0.08872342, -0.23723...","[-0.037886444, -0.14366937, 0.1765283, -0.3061..."
61343,61344,soP7Okyp0e1WdYxQC9IldpwnznT6V1,NaN,"[0.59241676, -0.06305879, 0.2820058, -0.444997...","[0.27924356, 0.25066462, 0.058347516, -0.12136..."
61344,61345,soP7Okyp0e1WdYxQC9IldpwnznT6V1,NaN,"[-0.056848947, -0.3328433, 0.07876696, -0.2370...","[0.011485679, -0.10086178, -0.30850887, -0.177..."


In [5]:
# Step 1: Reapply Grouping Logic to `df` (OpenAI DataFrame)
df['is_new_group'] = (
    (df['worker'] != df['worker'].shift()) |
    (df['newsID'] != df['newsID'].shift()) |
    (df['Article'] != df['Article'].shift())  # Cross-check for content changes
)
df['group_id'] = df['is_new_group'].cumsum()

In [6]:
# Step 2: Merge the Newly Grouped `df` with `grouped`
final_merged_df = pd.merge(
    df,
    grouped[['group_id', 'Article', 'Headline']],  # Select only necessary columns from grouped
    on='group_id',  # Merge based on group_id
    how='left'  # Ensure all rows in df are retained
)


In [7]:
final_merged_df

,Unnamed: 0,worker,confidence,newsID,Article_x,category,summary,Headline_x,is_new_group,group_id,Article_y,Headline_y
0,0,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,5.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,Fiance and I recently got infected with scabi...,19f with fiance 20m and roommate 19m- fiance a...,True,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
1,1,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,4.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,my fiance and I refuse to treat our room mate...,19f with fiance 20m and roommate 19m- fiance a...,False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
2,2,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,"fiancé and I contracted scabies, roommate ref...",19f with fiance 20m and roommate 19m- fiance a...,False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
3,3,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,"fiance and I are infected with scabies, room ...",19f with fiance 20m and roommate 19m- fiance a...,False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
4,4,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,"Fiance and I contracted scabies, roommate ref...",19f with fiance 20m and roommate 19m- fiance a...,False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
...,...,...,...,...,...,...,...,...,...,...,...,...
185240,185240,RMwrIV50cNusBthNvLs1wSNdqFpQAg,1.0,t3_kz1gv,Hello World of Reddit\n\nI'm a 21 year old fem...,AskReddit,I'm a 21year old female anemic taking iron su...,"I have iron deficient anemia, need help and/or...",False,61342,"[0.26689363, -0.29950368, 0.18191998, -0.29945...","[0.038773768, 0.12896419, 0.0573383, -0.503152..."
185241,185241,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,I don't know how to ask this without it soundi...,r/relationships,"Boyfriend talks very slowly, I feel frustrated...",Me [27 F] with my boyfriend [24 M] for four mo...,True,61343,"[0.11833226, -0.15423085, 0.08872342, -0.23723...","[-0.037886444, -0.14366937, 0.1765283, -0.3061..."
185242,185242,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,I honestly do not care anymore. \n\nExcluding ...,r/Advice,Medical field looks like its going to hell. Wh...,Almost every career in the medical field is sa...,True,61344,"[0.59241676, -0.06305879, 0.2820058, -0.444997...","[0.27924356, 0.25066462, 0.058347516, -0.12136..."
185243,185243,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,Someone I consider to be a best friend and I a...,r/relationship_advice,"Met a guy on reddit, became best friends, met,...",I'm [26/f] in US and my best friend [32/m] is ...,True,61345,"[-0.056848947, -0.3328433, 0.07876696, -0.2370...","[0.011485679, -0.10086178, -0.30850887, -0.177..."


In [8]:
# Step 3: Rename Columns for Clarity
final_merged_df.rename(columns={'Article': 'article_embed', 'Headline': 'headline_embed'}, inplace=True)

In [9]:
# Print all column names to check their actual names
print(final_merged_df.columns.tolist())


['Unnamed: 0', 'worker', 'confidence', 'newsID', 'Article_x', 'category', 'summary', 'Headline_x', 'is_new_group', 'group_id', 'Article_y', 'Headline_y']


In [10]:
# Drop the unwanted columns
final_merged_df = final_merged_df.drop(columns=['Article_x', 'Headline_x'])

# Rename the columns
final_merged_df = final_merged_df.rename(columns={'Article_y': 'Article', 'Headline_y': 'Headline'})



In [11]:
semi_merged_df = final_merged_df

In [12]:
semi_merger_df

NameError: name 'semi_merger_df' is not defined

In [23]:
# Save the updated DataFrame
output_path = r"C:\Users\Hemish Savaliya\Downloads\Embedding\article_headline_embed.csv"
semi_merged_df.to_csv(output_path, index=False)
print(f"Updated merged DataFrame saved successfully at {output_path}!")

Updated merged DataFrame saved successfully at C:\Users\Hemish Savaliya\Downloads\Embedding\article_headline_embed.csv!


In [24]:
import pandas as pd

# Define file paths
merged_df_path = r"C:\Users\Hemish Savaliya\Documents\Chunks_embed\updated_merged_df.csv"
semi_embed_df_path = r"C:\Users\Hemish Savaliya\Downloads\Embedding\article_headline_embed.csv"

# Load the DataFrames
merged_df = pd.read_csv(merged_df_path)
semi_embed_df = pd.read_csv(semi_embed_df_path)




In [25]:
semi_embed_df

,Unnamed: 0,worker,confidence,newsID,category,summary,is_new_group,group_id,Article,Headline
0,0,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,5.0,t3_4l0bal,relationship_advice,Fiance and I recently got infected with scabi...,True,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
1,1,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,4.0,t3_4l0bal,relationship_advice,my fiance and I refuse to treat our room mate...,False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
2,2,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,relationship_advice,"fiancé and I contracted scabies, roommate ref...",False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
3,3,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,relationship_advice,"fiance and I are infected with scabies, room ...",False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
4,4,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,relationship_advice,"Fiance and I contracted scabies, roommate ref...",False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
...,...,...,...,...,...,...,...,...,...,...
185240,185240,RMwrIV50cNusBthNvLs1wSNdqFpQAg,1.0,t3_kz1gv,AskReddit,I'm a 21year old female anemic taking iron su...,False,61342,"[0.26689363, -0.29950368, 0.18191998, -0.29945...","[0.038773768, 0.12896419, 0.0573383, -0.503152..."
185241,185241,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,r/relationships,"Boyfriend talks very slowly, I feel frustrated...",True,61343,"[0.11833226, -0.15423085, 0.08872342, -0.23723...","[-0.037886444, -0.14366937, 0.1765283, -0.3061..."
185242,185242,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,r/Advice,Medical field looks like its going to hell. Wh...,True,61344,"[0.59241676, -0.06305879, 0.2820058, -0.444997...","[0.27924356, 0.25066462, 0.058347516, -0.12136..."
185243,185243,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,r/relationship_advice,"Met a guy on reddit, became best friends, met,...",True,61345,"[-0.056848947, -0.3328433, 0.07876696, -0.2370...","[0.011485679, -0.10086178, -0.30850887, -0.177..."


In [26]:
merged_df

,group_id,worker,newsID,Article,Headline,embeddings_article,embeddings_headline
0,1,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",19f with fiance 20m and roommate 19m- fiance a...,"[-0.18161316215991974, -0.04418599233031273, 0...","[-0.46825119853019714, -0.19821323454380035, 0..."
1,2,dL2Aff39094mOhTKsLkJg8sCCFbirW,t3_4iqygu,My boyfriend and I have been together little o...,Me [17 F] with my boyfriend [17 M] He is being...,"[0.12550994753837585, -0.014912053942680359, -...","[-0.1613437831401825, 0.09565150737762451, -0...."
2,3,dL2Aff39094mOhTKsLkJg8sCCFbirW,t3_2dmjeu,"Sorry for the wall of text, it's hard to summa...",What should my next move be?,"[0.21124741435050964, -0.36937960982322693, 0....","[-0.2028527408838272, 0.1643262654542923, 0.01..."
3,4,dL2Aff39094mOhTKsLkJg8sCCFbirW,t3_2k3zi1,I share an apartment with my boyfriend. There ...,Revenge on a New Roommate,"[0.1966981291770935, -0.27256715297698975, -0....","[-0.1573682427406311, -0.09334687888622284, -0..."
4,5,RMwrIV50cNusBthNvLs1wSNdqFpQAg,t3_zjaee,So me and my girlfriend (both 22) have been to...,Girlfriend suffers from anxiety; after a crazy...,"[0.11704462021589279, -0.3707609176635742, 0.0...","[-0.20783931016921997, -0.27736032009124756, -..."
...,...,...,...,...,...,...,...
48139,48140,FCzllSEpfOHCBBEJqq4VeHRQR5JdoX,t3_1sve9o,"I am a 17 year old male and my two friends, le...","Going to a concert with two friends, but anoth...","[0.1315484642982483, -0.2159823626279831, -0.0...","[-0.06082131341099739, 0.1590406596660614, -0...."
48140,48141,ZzGCcAhvqF0HnKxNsUjtJFadcZdyZj,t3_1sve9o,"I am a 17 year old male and my two friends, le...","Going to a concert with two friends, but anoth...","[0.1315484642982483, -0.2159823626279831, -0.0...","[-0.06082131341099739, 0.1590406596660614, -0...."
48141,48142,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,t3_1sve9o,"I am a 17 year old male and my two friends, le...","Going to a concert with two friends, but anoth...","[0.1315484642982483, -0.2159823626279831, -0.0...","[-0.06082131341099739, 0.1590406596660614, -0...."
48142,48143,RMwrIV50cNusBthNvLs1wSNdqFpQAg,t3_1sve9o,"I am a 17 year old male and my two friends, le...","Going to a concert with two friends, but anoth...","[0.1315484642982483, -0.2159823626279831, -0.0...","[-0.06082131341099739, 0.1590406596660614, -0...."


In [27]:
import pandas as pd

# Load the DataFrames
merged_df_path = r"C:\Users\Hemish Savaliya\Documents\Chunks_embed\Merged_Fixed_Embeddings.csv"
semi_merged_df_path = r"C:\Users\Hemish Savaliya\Downloads\Embedding\article_headline_embed.csv"

merged_df = pd.read_csv(merged_df_path)
semi_merged_df = pd.read_csv(semi_merged_df_path)

# Ensure there are no duplicate rows in merged_df for the join keys
merged_df = merged_df.drop_duplicates(subset=['worker', 'confidence', 'newsID'])

# Merge based on worker, confidence, and newsID
final_merged_df = semi_merged_df.merge(
    merged_df[['worker', 'confidence', 'newsID', 'summary', 'embeddings']],  # Include embeddings column
    on=['worker', 'confidence', 'newsID'],
    how='left',  # Keep all rows from semi_merged_df
    suffixes=('_semi', '_merged')  # Add suffixes to avoid column name conflicts
)

# Replace the summary column in semi_merged_df with the one from merged_df
final_merged_df['summary'] = final_merged_df['summary_merged']  # Use the summary from merged_df
final_merged_df.drop(columns=['summary_semi', 'summary_merged'], inplace=True)  # Drop unnecessary columns

# Ensure the shape matches the original semi_merged_df
assert len(final_merged_df) == len(semi_merged_df), "Final DataFrame shape mismatch!"

# Save the final DataFrame
final_output_path = r"C:\Users\Hemish Savaliya\Downloads\Embedding\final_merged_embeddings_with_column.csv"
final_merged_df.to_csv(final_output_path, index=False)
print(f"Final merged DataFrame saved successfully at {final_output_path}!")


Final merged DataFrame saved successfully at C:\Users\Hemish Savaliya\Downloads\Embedding\final_merged_embeddings_with_column.csv!


In [28]:
merged_df

,Unnamed: 0,worker,confidence,newsID,Article,category,summary,Headline,embeddings
0,0,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,5.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,Fiance and I recently got infected with scabi...,19f with fiance 20m and roommate 19m- fiance a...,"[0.241518512, 0.0324364714, 0.150843814, 0.012..."
1,1,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,4.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,my fiance and I refuse to treat our room mate...,19f with fiance 20m and roommate 19m- fiance a...,"[0.283570379, 0.248709887, 0.0302230809, 0.101..."
2,2,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,"Recently, my fiance (20 m) and I (19f) moved ...",relationship_advice,"fiancé and I contracted scabies, roommate ref...",19f with fiance 20m and roommate 19m- fiance a...,"[0.146174684, 0.159519061, -0.0568171516, 0.08..."
6,6,dL2Aff39094mOhTKsLkJg8sCCFbirW,4.0,t3_4iqygu,My boyfriend and I have been together little o...,relationships,My boyfriend has been mean to me months after...,Me [17 F] with my boyfriend [17 M] He is being...,"[0.108367763, 0.186665282, -0.0800745338, -0.1..."
7,7,dL2Aff39094mOhTKsLkJg8sCCFbirW,5.0,t3_4iqygu,My boyfriend and I have been together little o...,relationships,I sexted someone and my boyfriend has been me...,Me [17 F] with my boyfriend [17 M] He is being...,"[0.0351026617, 0.10907612, 0.0297342725, -0.05..."
...,...,...,...,...,...,...,...,...,...
185237,49992,RMwrIV50cNusBthNvLs1wSNdqFpQAg,8.0,t3_4vsc62,"MY bf of 3 years are going through a timeoff, ...",relationships,"He said he wanted to come and walk the dog, p...","Me [24F] with my bf [24 M] 3 years, Am I wrong...","[0.094248116, 0.135131657, 0.187506959, 0.1242..."
185238,49993,RMwrIV50cNusBthNvLs1wSNdqFpQAg,6.0,t3_4vsc62,"MY bf of 3 years are going through a timeoff, ...",relationships,"He said he wanted to come and walk the dog, p...","Me [24F] with my bf [24 M] 3 years, Am I wrong...","[0.094248116, 0.135131657, 0.187506959, 0.1242..."
185239,49994,P2p07Up4eJyvxrrVYgwtb60krbFbxI,6.0,t3_4vsc62,"MY bf of 3 years are going through a timeoff, ...",relationships,"He said he wanted to come and walk the dog, p...","Me [24F] with my bf [24 M] 3 years, Am I wrong...","[0.094248116, 0.135131657, 0.187506959, 0.1242..."
185240,49995,P2p07Up4eJyvxrrVYgwtb60krbFbxI,9.0,t3_4vsc62,"MY bf of 3 years are going through a timeoff, ...",relationships,"He said he wanted to come and walk the dog, p...","Me [24F] with my bf [24 M] 3 years, Am I wrong...","[0.094248116, 0.135131657, 0.187506959, 0.1242..."


In [29]:
semi_merged_df

,Unnamed: 0,worker,confidence,newsID,category,summary,is_new_group,group_id,Article,Headline
0,0,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,5.0,t3_4l0bal,relationship_advice,Fiance and I recently got infected with scabi...,True,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
1,1,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,4.0,t3_4l0bal,relationship_advice,my fiance and I refuse to treat our room mate...,False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
2,2,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,relationship_advice,"fiancé and I contracted scabies, roommate ref...",False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
3,3,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,relationship_advice,"fiance and I are infected with scabies, room ...",False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
4,4,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,relationship_advice,"Fiance and I contracted scabies, roommate ref...",False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135..."
...,...,...,...,...,...,...,...,...,...,...
185240,185240,RMwrIV50cNusBthNvLs1wSNdqFpQAg,1.0,t3_kz1gv,AskReddit,I'm a 21year old female anemic taking iron su...,False,61342,"[0.26689363, -0.29950368, 0.18191998, -0.29945...","[0.038773768, 0.12896419, 0.0573383, -0.503152..."
185241,185241,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,r/relationships,"Boyfriend talks very slowly, I feel frustrated...",True,61343,"[0.11833226, -0.15423085, 0.08872342, -0.23723...","[-0.037886444, -0.14366937, 0.1765283, -0.3061..."
185242,185242,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,r/Advice,Medical field looks like its going to hell. Wh...,True,61344,"[0.59241676, -0.06305879, 0.2820058, -0.444997...","[0.27924356, 0.25066462, 0.058347516, -0.12136..."
185243,185243,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,r/relationship_advice,"Met a guy on reddit, became best friends, met,...",True,61345,"[-0.056848947, -0.3328433, 0.07876696, -0.2370...","[0.011485679, -0.10086178, -0.30850887, -0.177..."


In [30]:
final_merged_df

,Unnamed: 0,worker,confidence,newsID,category,is_new_group,group_id,Article,Headline,embeddings,summary
0,0,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,5.0,t3_4l0bal,relationship_advice,True,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135...","[0.241518512, 0.0324364714, 0.150843814, 0.012...",Fiance and I recently got infected with scabi...
1,1,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,4.0,t3_4l0bal,relationship_advice,False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135...","[0.283570379, 0.248709887, 0.0302230809, 0.101...",my fiance and I refuse to treat our room mate...
2,2,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,relationship_advice,False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135...","[0.146174684, 0.159519061, -0.0568171516, 0.08...","fiancé and I contracted scabies, roommate ref..."
3,3,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,relationship_advice,False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135...","[0.146174684, 0.159519061, -0.0568171516, 0.08...","fiancé and I contracted scabies, roommate ref..."
4,4,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,6.0,t3_4l0bal,relationship_advice,False,1,"[-0.18161316, -0.044185992, 0.42439264, -0.618...","[-0.4682512, -0.19821323, 0.35738698, -0.22135...","[0.146174684, 0.159519061, -0.0568171516, 0.08...","fiancé and I contracted scabies, roommate ref..."
...,...,...,...,...,...,...,...,...,...,...,...
185240,185240,RMwrIV50cNusBthNvLs1wSNdqFpQAg,1.0,t3_kz1gv,AskReddit,False,61342,"[0.26689363, -0.29950368, 0.18191998, -0.29945...","[0.038773768, 0.12896419, 0.0573383, -0.503152...","[0.164478287, 0.312750876, 0.172689959, -0.002...",I'm a 21year old female anemic taking iron su...
185241,185241,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,r/relationships,True,61343,"[0.11833226, -0.15423085, 0.08872342, -0.23723...","[-0.037886444, -0.14366937, 0.1765283, -0.3061...","[-0.0145652816, 0.284733742, 0.146655023, -0.0...","Boyfriend talks very slowly, I feel frustrated..."
185242,185242,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,r/Advice,True,61344,"[0.59241676, -0.06305879, 0.2820058, -0.444997...","[0.27924356, 0.25066462, 0.058347516, -0.12136...","[-0.0145652816, 0.284733742, 0.146655023, -0.0...","Boyfriend talks very slowly, I feel frustrated..."
185243,185243,soP7Okyp0e1WdYxQC9IldpwnznT6V1,2.0,NaN,r/relationship_advice,True,61345,"[-0.056848947, -0.3328433, 0.07876696, -0.2370...","[0.011485679, -0.10086178, -0.30850887, -0.177...","[-0.0145652816, 0.284733742, 0.146655023, -0.0...","Boyfriend talks very slowly, I feel frustrated..."


In [31]:
# Save the final DataFrame to a new file
final_output_path = r"C:\Users\Hemish Savaliya\Downloads\Embedding\final_merged_embeddings.csv"
final_merged_df.to_csv(final_output_path, index=False)
print(f"Final merged DataFrame saved successfully at {final_output_path}!")


Final merged DataFrame saved successfully at C:\Users\Hemish Savaliya\Downloads\Embedding\final_merged_embeddings.csv!
